In [9]:
# Imports
import pandas as pd
import numpy as np
import nltk
import utils as ul
import matplotlib.pyplot as plt
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\kwnka\\vs-code projects\\CompIntel\\project_A\\utils.py'>

In [10]:
df = pd.read_csv('iphi2802.csv', delimiter='\t')

ul.print_info_df(df)

# Create a new column 'mean_date' in the dataframe.  
df['mean_date'] = df[['date_min', 'date_max']].mean(axis=1)

stopwords = nltk.corpus.stopwords.words('greek') 
texts = ul.vectorize_inscriptions(df, stopwords, 8000)
dates = df['mean_date'].values.reshape(-1,1)

# Scale both input and output data
scaler = MinMaxScaler()
X = scaler.fit_transform(texts)
y = scaler.fit_transform(dates)

# Initialize 5-Fold Cross Validation.
fold_dataset = ul.split_to_folds(5, X, y)


Original shape of Dataset: (2802, 11)


Number of NULL values per column:id                0
text              0
metadata          0
region_main_id    0
region_main       0
region_sub_id     0
region_sub        0
date_str          0
date_min          0
date_max          0
date_circa        0
dtype: int64


Number of unique values per column:id                2802
text              2795
metadata          2530
region_main_id      14
region_main         14
region_sub_id       73
region_sub          71
date_str          1384
date_min           505
date_max           509
date_circa           2
dtype: int64



c:\Users\kwnka\miniconda3\envs\computIntel\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['δι', 'ἀλλ'] not in stop_words.
  warnings.warn(


The shape of the matrix is: (2802, 8000) 

The vocabulary created from the vectorizer is the following 
 ['αα', 'αβ', 'αββα', 'αββεος', 'αβλα', 'αβλαναθαναλβα', 'αβρασαξ', 'αβρομαχος', 'αβρομαχου', 'αγ', 'αγα', 'αγαθ', 'αγαθα', 'αγαθαι', 'αγαθαν', 'αγαθει', 'αγαθη', 'αγαθηι', 'αγαθην', 'αγαθιας', 'αγαθο', 'αγαθοι', 'αγαθοις', 'αγαθοκλευς', 'αγαθοκλη', 'αγαθοκλης', 'αγαθον', 'αγαθοπους', 'αγαθος', 'αγαθου', 'αγαθους', 'αγαθω', 'αγαθωι', 'αγαθων', 'αγαθωνι', 'αγαθωνος', 'αγαθωνυμωι', 'αγαλμ', 'αγαλμα', 'αγαλμασιν', 'αγγ', 'αγγραψαι', 'αγε', 'αγελαν', 'αγελαου', 'αγελας', 'αγεμονιδος', 'αγεν', 'αγεπολεως', 'αγηι', 'αγηιξενου', 'αγησανδρος', 'αγησαρχος', 'αγησαρχου', 'αγησιας', 'αγησικλευς', 'αγι', 'αγια', 'αγιαδας', 'αγιαν', 'αγιας', 'αγιε', 'αγιο', 'αγιος', 'αγιου', 'αγιω', 'αγιων', 'αγιωνος', 'αγκαιον', 'αγκυ', 'αγλαος', 'αγν', 'αγνη', 'αγνης', 'αγνον', 'αγνουσιος', 'αγνως', 'αγον', 'αγονσαν', 'αγοντι', 'αγορα', 'αγοραι', 'αγορακριτου', 'αγοραν', 'αγορανομησαντα', 'αγορανομοι', 'αγορανο

In [ ]:
# Train and evaluate various models 
mean_losses_tr = []
mean_losses_ts = []

mean_train_loss, mean_test_loss = train_model(250, 0.001, 0.9, fold_dataset, 32, 50, False, False, 0.0, 0.0)
mean_losses_tr.append(mean_train_loss)
mean_losses_ts.append(mean_test_loss)

mean_train_loss, mean_test_loss = train_model([500,300], 0.001, fold_dataset, 32, 50, False, False, 0.0, 0.0)
mean_losses_tr.append(mean_train_loss)
mean_losses_ts.append(mean_test_loss)

mean_train_loss, mean_test_loss = train_model([400,200,100], 0.001, fold_dataset, 32, 50, False, False, 0.0, 0.0)
mean_losses_tr.append(mean_train_loss)
mean_losses_ts.append(mean_test_loss)

mean_train_loss, mean_test_loss = train_model([500,300,100], 0.001, fold_dataset, 32, 50, False, False, 0.0, 0.0)
mean_losses_tr.append(mean_train_loss)
mean_losses_ts.append(mean_test_loss)

mean_train_loss, mean_test_loss = train_model([800,400,200], 0.001, fold_dataset, 32, 50, False, False, 0.0, 0.0)
mean_losses_tr.append(mean_train_loss)
mean_losses_ts.append(mean_test_loss)



In [ ]:
# Plot the training and validation losses for each one individually
plt.figure(figsize=(12, 4))

for i, (train_loss, test_loss) in enumerate(zip(mean_losses_tr, mean_losses_ts)):
    plt.plot(train_loss, label=f'Mean Train Loss Network {i + 1}')
    plt.plot(test_loss, linestyle='--', label=f'Mean Test Loss Network {i + 1}')

plt.legend(fontsize = 6)
plt.title('Mean Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# Run the best model of the above using the early stopping criterion and plot the loss per fold for all the epochs.

train_loss, test_loss = train_model([500,300,100], 0.001, fold_dataset, 32, 150, True, False, 0.0, 0.0)

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs for every Fold with Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# Test the effects of combinations of various learning rate and momentum values

final_means = []

train_loss, test_loss = train_model([500,300,100], 0.001, 0.2, fold_dataset, 32, 150, True, False, 0.0, 0.0)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Early Stopping, η=0.001, m=0.6')
plt.xlabel('Epochs')
plt.ylabel('Loss')

train_loss, test_loss = train_model([500,300,100], 0.001, 0.6, fold_dataset, 32, 150,True, False, 0.0, 0.0)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Early Stopping, η=0.001, m=0.6')
plt.xlabel('Epochs')
plt.ylabel('Loss')

train_loss, test_loss = train_model([500,300,100], 0.05, 0.6, fold_dataset, 32, 150, True, False, 0.0, 0.0)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Early Stopping, η=0.05, m=0.6')
plt.xlabel('Epochs')
plt.ylabel('Loss')

train_loss, test_loss = train_model([500,300,100], 0.1, 0.6, fold_dataset, 32, 150, True, False, 0.0, 0.0)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Early Stopping, η=0.1, m=0.6')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# Test the network using dropout regularization with various values for dropout rates for the input and hidden layers 
final_means = []

train_loss, test_loss = train_model([500,300,100], 0.001, 0.9, fold_dataset, 32, 150, have_callback=True, dropout=True, dropout_in=0.8, dropout_h=0.5)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Dropout regularization, r_in=0.8, r_h= 0.5')
plt.xlabel('Epochs')
plt.ylabel('Loss')

train_loss, test_loss = train_model([500,300,100], 0.001, 0.9, fold_dataset, 32, 150, have_callback=True, dropout=True, dropout_in=0.5, dropout_h=0.5)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Dropout regularization, r_in=0.5, r_h= 0.5')
plt.xlabel('Epochs')
plt.ylabel('Loss')

train_loss, test_loss = train_model([500,300,100], 0.05, 0.9, fold_dataset, 32, 150, have_callback=True, dropout=True, dropout_in=0.8, dropout_h=0.2)
final_means.append(mean_of_all(train_loss))

plt.figure(figsize=(12, 4))

for i, (tr_loss, ts_loss) in enumerate(zip(train_loss, test_loss)):
    plt.plot(tr_loss, label=f'Train Loss Fold {i + 1}')
    plt.plot(ts_loss, linestyle='--', label=f'Test Loss Fold {i + 1}')

plt.legend(fontsize = 6)
plt.title('Loss Over Epochs per Fold with Dropout regularization, r_in=0.8, r_h= 0.2')
plt.xlabel('Epochs')
plt.ylabel('Loss')